We want to do a book recommendation system based on the data we have cleaned before.
They are three types of recommender system:
* Collaborative filtering:This system matches persons with similar interests and provides recommendations based on this matching. It needs the historical activity of the users which we have.It can be user-based or item-based. In the first, we recommend items to a user that similar users have also liked. In the second, we recommend items based on the past ratings of similiar items by the user.

* Content base systems: They suggest similar items based on a particular item. This system uses item metadata. The general idea behind these recommender systems is that if a person liked a particular item, he or she will also like an item that is similar to it. The only metadata we have about the books in our dataset is the title, author and country wich is very little. A good metadata is the genres or the summary that sadly we haven't. This method is also computationally expensive which is another reason we are not doing it.

This system recommendation were mainly done with the helf of the book "Hands on recommendation systems with Python" by Rounak Banik 
https://learning.oreilly.com/library/view/hands-on-recommendation-systems/9781788993753/5f1269b2-2007-42a8-a900-bf255b86e64c.xhtml


We want to predict the rating given by a user to a book based on the book that he has already read and the books of user that have like the same books i.e similar users

# Loading the data previously cleaned

In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [47]:
path="D:/Workspace_Python/MLProject/MachineLearningDSTIProject/dataset/"
file="final_dataset.csv"
users_csv="cleaned_users.csv"
books_csv="cleaned_books.csv"
ratings_csv="cleaned_ratings.csv"

In [48]:
df = pd.read_csv(path+file, sep=";",on_bad_lines='warn', encoding="latin-1")

## Predict the age target of a book

# Recommender system

## Item-based ou user-based collaborative filtering ?

In [5]:
users = pd.read_csv(path+users_csv, sep=";",on_bad_lines='warn', encoding="latin-1")
users.shape

(278858, 5)

In [7]:
books = pd.read_csv(path+books_csv, sep=";",on_bad_lines='warn', encoding="latin-1")
books.shape

(271379, 5)

As we have approximatively as much users than books and we have huge disparity in the number of ratings between users as seen in the EDA, we choose to use the item-based collaborative filtering wich should also be more efficient

to review

## Item-based collaborative filtering

We load the data into the data structure of the surprise library

In [51]:
# Loads Pandas dataframe
from surprise import Dataset
from surprise import Reader
#we reduce the size of the dataset by removing the user that rated less than 10 books and books 
#that have been rated less than 100 times
df=df[df.userID.isin(df.userID.value_counts()[df.userID.value_counts()>10].index)][['userID','bookId','bookRating']]
df=df[df.bookId.isin(df.bookId.value_counts()[df.bookId.value_counts()>100].index)][['userID','bookId','bookRating']]
data = Dataset.load_from_df(df[['userID','bookId','bookRating']], Reader(line_format='user item rating', sep=";",rating_scale=(1, 10)))

In [53]:
##to see what is in the built in data of surprise
d=pd.DataFrame(data.__dict__['raw_ratings'], columns=['user_id','item_id','rating','timestamp'])
d.head()

,user_id,item_id,rating,timestamp
0,242,180,8.0,None
1,243,306,7.0,None
2,243,834,9.0,None
3,243,4825,9.0,None
4,254,3460,9.0,None


We benchmark the different models by using a cross validation after having set the the seed of the RNG to have reproducible experiments.

In [54]:
import random
import numpy as np

my_seed = 0
random.seed(my_seed)
np.random.seed(my_seed)

In [55]:
from surprise import SVD
from surprise import SVDpp
from surprise import SlopeOne
from surprise import NMF
from surprise import NormalPredictor
from surprise import KNNBaseline
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import BaselineOnly
from surprise import CoClustering
from surprise.model_selection import cross_validate
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

C:\Users\adrie\AppData\Local\Temp\ipykernel_21080\1018682062.py:21: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
C:\Users\adrie\AppData\Local\Temp\ipykernel_21080\1018682062.py:21: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
C:\Users\adrie\AppData\Local\Temp\ipykernel_21080\1018682062.py:21: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
C:\Users\adrie\AppData\Local\Temp\ipykernel_21080\1018682062.py:21: FutureWarning: The series.append

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


C:\Users\adrie\AppData\Local\Temp\ipykernel_21080\1018682062.py:21: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...


C:\Users\adrie\AppData\Local\Temp\ipykernel_21080\1018682062.py:21: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))


...

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


C:\Users\adrie\AppData\Local\Temp\ipykernel_21080\1018682062.py:21: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
C:\Users\adrie\AppData\Local\Temp\ipykernel_21080\1018682062.py:21: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))


Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


C:\Users\adrie\AppData\Local\Temp\ipykernel_21080\1018682062.py:21: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))


,test_rmse,fit_time,test_time
Algorithm,,,
SVD,1.625641,0.092082,0.010992
BaselineOnly,1.628368,0.012713,0.007202
KNNBaseline,1.662919,0.183550,0.187955
SVDpp,1.680837,0.095030,0.029158
KNNBasic,1.738697,0.169970,0.157416
CoClustering,1.844757,0.411104,0.007960
KNNWithMeans,1.853848,0.213422,0.164554
SlopeOne,1.869185,0.040720,0.013561
KNNWithZScore,1.890926,0.342794,0.178875


BaselineOnly and SVD give us the best although BaselineOnly also give us the best time, so it' is this one we should use

In [56]:
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)

Using ALS
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([1.62038706, 1.59679814, 1.59217462, 1.6297444 , 1.66474006]),
 'fit_time': (0.009596109390258789,
  0.008472919464111328,
  0.007867813110351562,
  0.007852554321289062,
  0.006787300109863281),
 'test_time': (0.007916450500488281,
  0.0073468685150146484,
  0.007341146469116211,
  0.006822347640991211,
  0.007322072982788086)}

In [57]:
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=0.25)
algo = BaselineOnly(bsl_options=bsl_options)
predictions = algo.fit(trainset).test(testset)

Estimating biases using als...


Finally, we measure the accuracy of the ratings we have calculated

In [58]:
accuracy.rmse(predictions)
accuracy.mse(predictions)
accuracy.mae(predictions)
accuracy.fcp(predictions)

RMSE: 1.6762
MSE: 2.8096
MAE:  1.2918
FCP:  0.5344


0.5344255900136532

We tune the algorithm parameters by using grid search:

In [ ]:
from surprise.model_selection import GridSearchCV
to_do
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

We can now do some prediction:

In [1]:
#from neo4j import GraphDatabase
#driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "DSTI2023!!"))